In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/Shareddrives/CSE573/preprocessed
%ls

/content/drive/Shareddrives/CSE573/preprocessed
AaplAmzn_NewsDateDirection.csv
Aapl_NewsPriceHistDirection.csv
Aapl_SentimentAggMeanPriceHistDirection.csv
Aapl_SentimentAggSumPriceHistDirection.csv
Amzn_NewsPriceHistDirection.csv
Amzn_SentimentAggMeanPriceHistDirection.csv
Amzn_SentimentAggSumPriceHistDirection.csv
amzn_title_processedText_direction.csv
amzn_title_processedText_direction_date.csv
amzn_title_text_label.csv


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

import requests
import io

In [ ]:
df_amzn = pd.read_csv('amzn_title_processedText_direction_date.csv')


In [ ]:
df_amzn.head()

,title,text,direction,text_without_stopwords,processedText,prediction_date
0,6 things to know before the opening bell,click chart for more in-depth data. 1. big tec...,-1,click chart in-depth data 1. big tech senate b...,click chart in-depth data 1. big tech senate b...,2018-09-05
1,"Netflix And The Competition - Netflix, Inc. (N...",investment thesis netflix ( nflx ) is a remark...,-1,investment thesis netflix nflx remarkable serv...,investment thesis netflix nflx remarkable serv...,2018-09-05
2,Pandora Stock Gets a Boost — Buy Now,mannkind surges more than 50% >>> read more\nb...,-1,mannkind surges 50 read breaking news sponsore...,surges 50 read breaking news sponsored home st...,2018-09-05
3,MARKET SNAPSHOT: Tech Tumble Weighs On Stocks;...,"by mark decambre, marketwatch , ryan vlastelic...",-1,mark decambre marketwatch ryan vlastelica tech...,mark decambre marketwatch ryan vlastelica tech...,2018-09-05
4,"New Month, Same Old Story: Trade Fears Persist...","it may be a new month, but the same trade conc...",-1,may new month trade concerns haunted market ye...,may new month trade concerns market year remai...,2018-09-05


In [ ]:
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer


In [ ]:
#title_amzn=df_amzn['title'].tolist()
text_amzn=df_amzn['processedText'].tolist()
label_amzn=df_amzn['direction'].to_numpy()


#print(title_amzn[0])
print(text_amzn[0])
print(label_amzn[0])

click chart in-depth data 1. big tech senate big tech executives coming capitol hill wednesday testify election meddling facebook fb coo sheryl sandberg jack dorsey ceo twitter twtr appear senate intelligence committee discuss actions companies taken foreign influence campaigns targeting 2018 midterm elections committee requested high-profile executive google owns youtube also appear testify google googl parent company alphabet however declined send one top people 2. emerging markets fears emerging markets currencies tumbled dollar wednesday mexican south indian among biggest losers rising us interest rates political global trade war recent months exposed multiple emerging markets turkey argentina hardest hit investors worried adding fears markit index measures emerging markets manufacturing fell fresh low august watch cnnmoney now today p.m. et 3. global stocks overview us stock futures pointing lower european markets opened down exchanges london frankfurt paris dropping roughly 0.5 a

In [ ]:
class_label_amzn = pd.get_dummies(label_amzn).values

In [ ]:
text_tokenized_amzn, text_tokenizer_amzn=tokenize(text_amzn)
text_padded_amzn=pad_sequences(text_tokenized_amzn, padding='post')

In [ ]:
#print(title_tokenized_amzn[0])
print(text_padded_amzn[0])

[480 247 481 ...   0   0   0]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(text_padded_amzn, class_label_amzn, test_size=0.2, random_state=56)

In [ ]:
TrainX = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
TestX = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
TrainX = TrainX.astype('float64')
TestX = TestX.astype('float64')

In [ ]:
TrainX[0:5]

array([[[   3.,    6.,   34., ...,    0.,    0.,    0.]],

       [[  99.,   16., 2002., ...,    0.,    0.,    0.]],

       [[2353., 1771.,   66., ...,    0.,    0.,    0.]],

       [[  30.,   32.,    5., ...,    0.,    0.,    0.]],

       [[   8.,  456., 1393., ...,    0.,    0.,    0.]]])

In [ ]:
TrainX.shape

(15262, 1, 12487)

In [ ]:
Y_train.shape

(15262, 2)

In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import Recall, Precision

In [ ]:
layers = [LSTM(128,input_shape=(1,TrainX.shape[2]),dropout=0.2), Dense(2,activation='sigmoid')]
model = tf.keras.models.Sequential(layers)
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(),metrics=['accuracy', Recall(), Precision()])
model.fit(TrainX, Y_train, epochs=40, batch_size=64, verbose=2,validation_split=0.2)

Epoch 1/40
191/191 - 16s - loss: 0.7108 - accuracy: 0.5021 - recall_9: 0.1851 - precision_7: 0.5097 - val_loss: 0.7010 - val_accuracy: 0.5038 - val_recall_9: 0.1661 - val_precision_7: 0.4975 - 16s/epoch - 83ms/step
Epoch 2/40
191/191 - 12s - loss: 0.6995 - accuracy: 0.5065 - recall_9: 0.1927 - precision_7: 0.5043 - val_loss: 0.7002 - val_accuracy: 0.5156 - val_recall_9: 0.4690 - val_precision_7: 0.5046 - 12s/epoch - 62ms/step
Epoch 3/40
191/191 - 14s - loss: 0.6971 - accuracy: 0.5128 - recall_9: 0.3405 - precision_7: 0.5074 - val_loss: 0.6995 - val_accuracy: 0.4900 - val_recall_9: 0.1042 - val_precision_7: 0.4804 - 14s/epoch - 74ms/step
Epoch 4/40
191/191 - 12s - loss: 0.6951 - accuracy: 0.5118 - recall_9: 0.1184 - precision_7: 0.5081 - val_loss: 0.6966 - val_accuracy: 0.5077 - val_recall_9: 0.1310 - val_precision_7: 0.4884 - 12s/epoch - 62ms/step
Epoch 5/40
191/191 - 12s - loss: 0.6943 - accuracy: 0.5139 - recall_9: 0.1420 - precision_7: 0.5120 - val_loss: 0.6978 - val_accuracy: 0.513

In [ ]:
eva = model.evaluate(TestX, Y_test, verbose=2)

120/120 - 2s - loss: 0.6942 - accuracy: 0.5113 - recall_9: 0.0073 - precision_7: 0.4242 - 2s/epoch - 15ms/step


In [ ]:
f1 = 2 * ((eva[2]*eva[3])/(eva[2]+eva[3]))

In [ ]:
f1

0.014425554256822926

In [ ]:
model.save("../models/lstm_amzn_fulltext")